In [87]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import logging
from datetime import datetime, timezone,timedelta
from typing import List

from redis.asyncio import Redis

from app.database_redis.connection import get_redis_client
from app.database_redis.dals.connection_dal import ConnectionDAL
from app.services.apis.streamqueue_service.client import StreamQueueServiceAPI
from app.services.apis.streamqueue_service.schemas import AudioChunkInfo, ExistingConnectionInfo
from app.services.audio.redis import Connection, Diarizer, Meeting, Transcriber
from app.settings import settings

from app.services.audio.audio import AudioSlicer

from app.services.audio.redis import Diarisation, Diarizer, Meeting, best_covering_connection,connection_with_minimal_start_greater_than_target

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

In [89]:
current_time = datetime.now(timezone.utc)

In [99]:
meeting = Meeting(redis_client,'csx-mskk-qyk')
await meeting.load_from_redis()

In [100]:
meeting.diarizer_seek_timestamp

datetime.datetime(2024, 5, 6, 11, 48, 34, 356676, tzinfo=tzutc())

In [101]:
meeting.diarizer_seek_timestamp = current_time - timedelta(days=30)

In [102]:
meeting.transcriber_seek_timestamp = current_time - timedelta(days=30)

In [103]:
await meeting.update_redis()

In [104]:
connections = await meeting.get_connections()

In [105]:
len(connections)

11

In [106]:
connection = best_covering_connection(meeting.diarizer_seek_timestamp, current_time, connections) #

In [107]:
connection.end_timestamp

datetime.datetime(2024, 5, 31, 9, 0, 58, 580873, tzinfo=tzutc())

In [108]:
connection.start_timestamp

datetime.datetime(2024, 5, 31, 8, 51, 17, 295580, tzinfo=tzutc())

In [109]:
meeting.diarizer_seek_timestamp

datetime.datetime(2024, 5, 6, 11, 48, 34, 356676, tzinfo=datetime.timezone.utc)

In [110]:
connection.end_timestamp-meeting.diarizer_seek_timestamp

datetime.timedelta(days=24, seconds=76344, microseconds=224197)

In [111]:
connections

In [120]:
diarizer_step = 0
transcriber_step=0

In [115]:
current_time = datetime.now(timezone.utc)
segment_start_timestamp = current_time
if (current_time - meeting.diarizer_last_updated_timestamp).seconds > diarizer_step:
    print("diarizer added")
    diarizer = Diarizer(redis_client)
    await diarizer.add_todo(meeting.meeting_id)
    await meeting.update_diarizer_timestamp(
        segment_start_timestamp, diarizer_last_updated_timestamp=current_time
    )

diarizer added


In [122]:
meeting.transcriber_last_updated_timestamp

datetime.datetime(2024, 6, 5, 11, 49, 19, 741695, tzinfo=datetime.timezone.utc)

In [145]:
current_time = datetime.now(timezone.utc)
segment_start_timestamp = current_time
if (current_time - meeting.transcriber_last_updated_timestamp).seconds > transcriber_step:
    print("transcriber added")
    transcriber = Transcriber(redis_client)
    await transcriber.add_todo(meeting.meeting_id)
    await meeting.update_transcriber_timestamp(
        segment_start_timestamp, transcriber_last_updated_timestamp=current_time
    )

In [155]:
connid = 'e5e24183-6872-41e1-95c5-7d67f3fbc265'

In [156]:
path = f"/audio/{connid}.webm"

In [158]:
seek = 57.807868

In [159]:
max_length = 240

In [161]:
audio_slicer = await AudioSlicer.from_ffmpeg_slice(path, 0, max_length)

In [163]:
audio_slicer.audio.duration_seconds

57.816